In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Therefore, we have two sets:

- `X_train`: for each example, contains 3 features:
            - Ear Shape (1 if pointy, 0 otherwise)
            - Face Shape (1 if round, 0 otherwise)
            - Whiskers (1 if present, 0 otherwise)
            
- `y_train`: whether the animal is a cat
            - 1 if the animal is a cat
            - 0 otherwise

In [3]:
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [4]:
def entropy(p):
    if p == 0 or p == 1:
        return 0
    else:
        return -p * np.log2(p) - (1- p)*np.log2(1 - p)
    
print(entropy(0.5))

1.0


In [44]:
#To illustrate, let's compute the information gain 
#if we split the node for each of the features. 
def split_indices(X, index_feature):
    """Given a dataset and a index feature, return two lists for the two split nodes, the left node has the animals that have 
    that feature = 1 and the right node those that have the feature = 0 
    index feature = 0 => ear shape
    index feature = 1 => face shape
    index feature = 2 => whiskers
    """
    left_indices = []
    right_indices = []
    for i,x in enumerate(X):
        if x[index_feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

In [6]:
split_indices(X_train, 0)

([0, 3, 4, 5, 7], [1, 2, 6, 8, 9])

In [7]:
def weighted_entropy(X,y,left_indices,right_indices):
    """
    This function takes the splitted dataset, the indices we chose to split and returns the weighted entropy.
    """
    w_left = len(left_indices)/len(X)
    w_right = len(right_indices)/len(X)
    p_left = sum(y[left_indices])/len(left_indices)
    p_right = sum(y[right_indices])/len(right_indices)
    
    weighted_entropy = w_left * entropy(p_left) + w_right * entropy(p_right)
    return weighted_entropy

In [8]:
left_indices, right_indices = split_indices(X_train, 0)
weighted_entropy(X_train, y_train, left_indices, right_indices)

0.7219280948873623

In [9]:
def information_gain(X, y, left_indices, right_indices):
    """
    Here, X has the elements in the node and y is theirs respectives classes
    """
    p_node = sum(y)/len(y)
    h_node = entropy(p_node)
    w_entropy = weighted_entropy(X,y,left_indices,right_indices)
    return h_node - w_entropy

In [10]:
information_gain(X_train, y_train, left_indices, right_indices)

0.2780719051126377

In [19]:

def chosing_feature_for_node(features):
    i_gains = np.zeros(len(features))
    for i, feature_name in enumerate(features):
        left_indices, right_indices = split_indices(X_train, i)
        i_gain = information_gain(X_train, y_train, left_indices, right_indices)
        i_gains[i] = i_gain
        print(f"Feature: {feature_name}, information gain if we split the root node using this feature: {i_gain:.2f}")
    chosed_feature = np.argmax(i_gains)
    print(f"Choosed Feature: {chosed_feature} : {features[chosed_feature]}")
    return chosed_feature


Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12
Coosed Feature: 0 : Ear Shape


0

In [54]:
def prepare_train_set(X_train, y_train, indices):
    result_X_train = []
    for val in indices:
        result_X_train.append(X_train[val])
        
    result_y_train = []
    for val in indices:
        result_y_train.append(y_train[val])
    return result_X_train, result_y_train

def build_tree_recursive(X_train, y_train, features, tree, is_right = 0, max_depth=2, current_depth=0):
    print(f"Current Level: {current_depth}")
    if current_depth >= max_depth:
        print(f"Level completed by max depth!")
        return;
    chosed_feature = chosing_feature_for_node(features)
    left_indices, right_indices = split_indices(X_train, chosed_feature)
    tree[current_depth][is_right] = {features[chosed_feature]:[left_indices, right_indices]}
    print(f"Tree: {tree}")
    del(features[chosed_feature])
    
    if len(left_indices) >= 0:        
        left_X_train, left_y_train = prepare_train_set(X_train, y_train, left_indices)
        build_tree_recursive(left_X_train, left_y_train, features, tree, 0, max_depth = 2, current_depth=current_depth+1)
    else:
        print(f"Level completed by reaching 100% class!")
        return;
    if len(right_indices) >= 0:
        right_X_train, right_y_train = prepare_train_set(X_train, y_train, right_indices)
        build_tree_recursive(right_X_train, right_y_train, features, tree, 1, max_depth = 2, current_depth=current_depth+1)
    else:
        print(f"Level completed by reaching 100% class!")
        return;
    
features = ['Ear Shape', 'Face Shape', 'Whiskers']
tree = [[0, 0], [0, 0], [0, 0]]
build_tree_recursive(X_train, y_train, features, max_depth=2, current_depth=0, tree = tree)

Current Level: 0
Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12
Coosed Feature: 0 : Ear Shape
Tree: [[{'Ear Shape': [[0, 3, 4, 5, 7], [1, 2, 6, 8, 9]]}, 0], [0, 0], [0, 0]]
Current Level: 1
Feature: Face Shape, information gain if we split the root node using this feature: 0.28
Feature: Whiskers, information gain if we split the root node using this feature: 0.03
Coosed Feature: 0 : Face Shape
Tree: [[{'Ear Shape': [[0, 3, 4, 5, 7], [1, 2, 6, 8, 9]]}, 0], [{'Face Shape': [[0, 1, 2, 3, 4], []]}, 0], [0, 0]]
Current Level: 2
Level completed by max depth!
Current Level: 2
Level completed by max depth!
Current Level: 1
Feature: Whiskers, information gain if we split the root node using this feature: 0.28
Coosed Feature: 0 : Whiskers
Tree: [[{'Ear Shape': [[0, 3, 4, 5, 7], [1, 